In [1]:
CIFAR = 'cifar-10-batches-py/'
import numpy as np

In [2]:
def unpickle(file):
    import pickle
    with open(file,'rb') as fo:
        cifar_dict = pickle.load(fo,encoding='bytes')
    return cifar_dict

In [3]:
dirs = ['batches.meta','data_batch_1','data_batch_2','data_batch_3','data_batch_4','data_batch_5','test_batch']

all_data = [0,1,2,3,4,5,6]

for i, direc in zip(all_data,dirs):
    all_data[i] = unpickle(CIFAR+direc)

In [4]:
batch_meta = all_data[0]
data_batch1 = all_data[1]
data_batch2 = all_data[2]
data_batch3 = all_data[3]
data_batch4 = all_data[4]
data_batch5 = all_data[5]
test_batch = all_data[6]

In [5]:
def one_hot_encode(vec,vals=10):
    
    n = len(vec)
    out = np.zeros((n,vals))
    out[range(n), vec] = 1
    return out

In [5]:
all_train_batches = [data_batch1,data_batch2,data_batch3,data_batch4,data_batch5]
test_batch1 = [test_batch]

training_image = np.vstack([d[b"data"] for d in all_train_batches])
train_len = len(training_image)
training_image = training_image.reshape(train_len,3*32*32)
training_label = np.hstack([d[b"labels"] for d in all_train_batches])
training_label = training_label.reshape(50000)

In [6]:
training_label.shape

(50000,)

In [7]:
test_image = np.vstack([d[b"data"] for d in test_batch1])
test_len = len(test_image)
test_image = test_image.reshape(test_len,3*32*32)
test_label = np.hstack([d[b"labels"] for d in test_batch1])
test_label = test_label.reshape(10000)

In [8]:
from sklearn.decomposition import PCA

pca = PCA(.90)
X_train = pca.fit_transform(training_image)
X_test = pca.transform(test_image)

In [10]:
X_test.shape

(10000, 99)

In [12]:
import pickle
filename = 'XtrainPCA50000.sav'
pickle.dump(X_train, open(filename, 'wb'))

In [13]:
filename = 'XtestPCA10000.sav'
pickle.dump(X_test, open(filename, 'wb'))

In [ ]:
from sklearn.model_selection import GridSearchCV 
from sklearn.svm import SVC  
# defining parameter range 
param_grid = {'C': [0.1, 1, 10, 50],  
              'kernel': ['linear']}  
  
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3,cv=5,n_jobs=-1) 
  
# fitting the model for grid search 
grid.fit(X_train, training_label) 

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


In [ ]:
sv = grid.best_estimator_.fit(X_train,training_label)
filename = 'SVM50000pca.sav'
pickle.dump(sv, open(filename, 'wb'))

In [ ]:
print(grid.best_params_) 


In [15]:
print(grid.best_estimator_) 

array([6, 9, 9, ..., 9, 1, 1])

In [ ]:
grid_trainpredict = grid.predict(X_train)

In [ ]:
grid_predictions = grid.predict(X_test) 

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(test_label, grid_predictions))

In [ ]:
print(classification_report(test_label, grid_predictions))